Below is setup for Drake

In [0]:
try:
    import pydrake
    import underactuated
except ImportError:
    !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
    from jupyter_setup import setup_underactuated
    setup_underactuated()

# Setup matplotlib.
from IPython import get_ipython
if get_ipython() is not None: get_ipython().run_line_magic("matplotlib", "inline")

In [0]:
# python libraries
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
import sys 
import platform
import pydrake
from pydrake.solvers import branch_and_bound

# pydrake imports
from pydrake.all import (MultibodyPlant, Parser, DiagramBuilder, Simulator,
                         PlanarSceneGraphVisualizer, SceneGraph, TrajectorySource,
                         SnoptSolver, MultibodyPositionToGeometryPose, PiecewisePolynomial,
                         MathematicalProgram, JacobianWrtVariable, eq, LinearQuadraticRegulator,
                         Linearize, AddMultibodyPlantSceneGraph, VectorSystem, le, ge, OsqpSolver)
# underactuated imports
from underactuated import FindResource

In [6]:
#clears file and downloads urdf to have it be updated
get_ipython().system(u"rm -rf /opt/hopper")
if platform.system() == "Darwin":
  get_ipython().system(u"if [ ! -d '/opt/hopper' ]; then git clone https://github.com/Olliebear/underactuated_final_project.git /opt/hopper; fi")
elif platform.linux_distribution() == ("Ubuntu", "18.04", "bionic"):
  get_ipython().system(u"if [ ! -d '/opt/hopper' ]; then git clone https://github.com/Olliebear/underactuated_final_project.git /opt/hopper; fi" )
else:
  assert False, "Unsupported platform"
sys.path.append("/opt/hopper")

Cloning into '/opt/hopper'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 28 (delta 13), reused 22 (delta 7), pack-reused 0
Unpacking objects: 100% (28/28), done.


##Define the problem's parameters:
x0, y0 are coordinates of the foot of the 2d hopper.

th1 (theta1) is the angle between the foot and the upright, th2 is angle between bode and upright.

z0 is the distance from the hip pin joint to the foot. 

uT is torque input at pinjoint connecting leg and body. uX is length actuation on the spring in the leg. 

###Other parameters:
Spring length K_length= z0 - uX.

m1,I1,r1 = mass, inertia and radius from toe to C.O.M. for the leg

m2,I2,r2 = mass, inertia and radius from hip to C.O.M. for the body

##While on the ground:
For now, assume infinite friction. Can add friction cone constraint later, or add more modes

###Define Some System Constants
Order of states: x0, y0, th1, z0, th2

In [0]:
x_initial=[0, 3, 0, 0, 0, 0, 0, 0, 0, 0]


# weight matrices for the lqr controller
Q = np.eye(10)
R = np.eye(4)

# **Some info here**
In general, this code is a bit of a crap heap. Sorry. Lots of things/ideas going on here that I'm working on. The next few blocks though are the main part I think.

The problem is I need a better solver to actually solve my mathematical program, but I'm not sure how to get that onto this colab machine remotely. I'd imagine there's licensing issues. 

Also, it would be difficult for me to install drake on my personal computer because it would involve updating my OS (to get python 3.8)

In [36]:

builder=DiagramBuilder()
hopper, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
Parser(hopper).AddModelFromFile('/opt/hopper/hopper.urdf')
hopper.Finalize()
hopper=hopper.ToAutoDiffXd()

nq=hopper.num_positions()
print(nq)
nf=4 # 2 inputs ux ut and 2 contact forces

context=hopper.CreateDefaultContext()
context.SetTime(0)


5


In [0]:
def manipulator_equations(vars):
    
    # split input vector in subvariables
    # configuration, velocity, acceleration, stance-foot force
    assert vars.size == 3 * nq + nf
    split_at = [nq, 2 * nq, 3 * nq , 3*nq+2]
    q, qd, qdd, u, f = np.split(vars, split_at)
    
    u[0]=get_spring_force(q[3],u[0])

    # set compass gait state
    context = hopper.CreateDefaultContext()
    hopper.SetPositions(context, q)
    hopper.SetVelocities(context, qd)
    
    # matrices for the manipulator equations
    M = hopper.CalcMassMatrixViaInverseDynamics(context)
    Cv = hopper.CalcBiasTerm(context)
    tauG = hopper.CalcGravityGeneralizedForces(context)
    
    # Jacobian of the stance foot
    Jf = get_foot_jacobian(hopper, context)
    Jh = get_hip_jacobian(hopper, context)
    Jx = get_slide_jacobian(hopper,context)
    
    # return violation of the manipulator equations
    return M.dot(qdd) + Cv - tauG - Jf.T.dot(f) - Jh.T.dot(u[1]) - Jx.T.dot(u[0])

In [0]:
def foot_height(q):
    
    # get reference frames for the swing leg and the ground
    leg_frame = hopper.GetBodyByName('leg').body_frame()
    ground_frame = hopper.GetBodyByName('ground').body_frame()
    
    # position of the swing foot in ground coordinates
    context = hopper.CreateDefaultContext()
    hopper.SetPositions(context, q)
    swing_foot_position = hopper.CalcPointsPositions(
        context,
        leg_frame,
        np.array([0,0,0]), #position of foot relative to leg
        ground_frame
    )
    ground_h=get_ground_height(q[0])
    # return only the coordinate z
    # (distance normal to the ground)
    return swing_foot_position[-1]-ground_h

In [0]:
def get_foot_jacobian(hopper, context):
    
    # get reference frames for the given leg and the ground
    leg_frame = hopper.GetBodyByName('leg').body_frame()
    ground_frame = hopper.GetBodyByName('ground').body_frame()

    # compute Jacobian matrix
    J = hopper.CalcJacobianTranslationalVelocity(
        context,
        JacobianWrtVariable(0),
        leg_frame,
        np.array([0,0,0]),
        ground_frame,
        ground_frame
    )
    
    # discard y components since we are in 2D
    return J[[0, 2]]

In [0]:
def get_hip_jacobian(hopper, context):
    
    # get reference frames for the given leg and the ground
    body_frame = hopper.GetBodyByName('body').body_frame()
    slide_frame = hopper.GetBodyByName('slide').body_frame()
    ground_frame = hopper.GetBodyByName('ground').body_frame()

    # compute Jacobian matrix
    J = hopper.CalcJacobianAngularVelocity(
        context,
        JacobianWrtVariable(0),
        body_frame,
        slide_frame,
        ground_frame
    )
    print("Get hip jacobian. J:")
    print(J)
    # discard y components since we are in 2D
    #not sure if this is correct
    print("J[[0,2]]:")
    print (J[[0, 2]])
    return J[[0, 2]]

In [0]:
def get_ground_height(x):
  #use this to get the height at some position x. can change to have variable terrain
  return 0

In [0]:
def get_slide_jacobian(hopper, context):
    
    # get reference frames for the given leg and the ground
    slide_frame = hopper.GetBodyByName('slide').body_frame()
    leg_frame = hopper.GetBodyByName('leg').body_frame()
    ground_frame = hopper.GetBodyByName('ground').body_frame()


    # compute Jacobian matrix
    J = hopper.CalcJacobianTranslationalVelocity(
        context,
        JacobianWrtVariable(0),
        slide_frame,
        np.array([0,0,0]),
        leg_frame,
        ground_frame
    )
    print("slide jacobian J:")
    print(J)
    #not sure if this is correct

    # discard y components since we are in 2D
    return J[[0, 2]]

In [85]:
prog=MathematicalProgram()
N=50
dt=0.01
time_steps=int(N/dt)

q=prog.NewContinuousVariables(N+1,5,'q')
qd=prog.NewContinuousVariables(N+1,5,'qd')
qdd=prog.NewContinuousVariables(N+1,5,'qdd')
lambdas=prog.NewContinuousVariables(N,2,'lambdas')
uX=prog.NewContinuousVariables(N,2,'uX')
uT=prog.NewContinuousVariables(N,1,'uT')
grounded=prog.NewBinaryVariables(N,1,'grounded')

M=1000000

#print(state[0])


for t in range(N):

  prog.AddLinearConstraint(eq(q[t+1] , q[t] + dt * qd[t+1]))
  prog.AddLinearConstraint(eq(qd[t+1] , qd[t] + dt * qdd[t]))


  var=np.concatenate((q[t],qd[t],qdd[t],uX[t],uT[t],lambdas[t]))
  prog.AddConstraint(manipulator_equations, lb=[0]*nq, ub=[0]*nq, vars=var) #dynamics constraint
  
  #y force is > 0
  prog.AddLinearConstraint(lambdas[t,1]>=0)

  #y0 is always above ground. using get_ground_height so that we can change ground geometry
  #prog.AddConstraint(foot_height,lb=[0],ub=[(1-grounded[t])*M],vars=q[t])

  #below is complimentarity constraint
  prog.AddLinearConstraint(le(lambdas[t,0],grounded[t]*M))
  prog.AddLinearConstraint(ge(lambdas[t,0],-1*grounded[t]*M))

  prog.AddLinearConstraint(le(lambdas[t,1],grounded[t]*M))
  prog.AddLinearConstraint(le(q[t,1]-get_ground_height(q[t,0]),(1-grounded[t])*M))
  
  #add controller cost
  prog.AddCost(uX[t].dot(uX[t]))
  prog.AddCost(uT[t].dot(uT[t]))
  prog.AddCost(q[t].dot(q[t]))

solver=SnoptSolver()
result=solver.Solve(prog)

#bb = branch_and_bound.MixedIntegerBranchAndBound(prog, OsqpSolver().solver_id())
#result = bb.Solve()

ValueError: ignored

In [0]:
def get_spring_force(z0,uX):
  k=1000 #spring constant
  kL=1 #relaxed length

  return(-k*(kL-z0+uX))

Ok stuff below this was me trying things out

In [0]:
class Spring(VectorSystem):
    def __init__(self):
        
        # 5 inputs: hopper state and spring preload delta
        # 1 output: spring force
        VectorSystem.__init__(self, 10,  1)
        
        # spring stiffness
        self.k = 1000

    # note that this function is called at each time step
    def DoCalcVectorOutput(self, context, state_delta, unused, spring_force):
        
        # set the output of the spring block to
        # the value of the elastic force
        l = state_delta[3]
        delta = state_delta[10]
        spring_force[:] = [- self.k * (l + delta)]

This next function calculates the derivative of the state variables by running a solver on the dynamics equations. 

Because the dynamics change whether the robot is touching the ground or not, we keep track of a boolean "in_air" that tells us whether we are in the air or not.

Its defined as a function separate from my Hopper_Dynamics Class below to let me call it as a constraint for optimization

In [0]:
def calc_dstate(state,uX,uT,lam): 
  #uses solver to solve dyanmics equations to get derivative of state. dx/dt

  #define state variables
  x0=state[0]
  y0=state[1]
  th1=state[2]
  z0=state[3]
  th2=state[4]
  #and their derivatives
  dx0=state[5]
  dy0=state[6]
  dth1=state[7]
  dz0=state[8]
  dth2=state[9]

  #get contact forces
  Fx=lam[0]
  Fy=lam[1]

  #define constants
  r1=0.5
  r2=0.7
  m1=10
  m2=10
  I1=10
  I2=10
  k=1000
  kL=1 #relaxed length
  g=9.81

  #intermediates:
  W=z0-r1
  s1=pydrake.symbolic.sin(th1)
  s2=pydrake.symbolic.sin(th2)
  c1=pydrake.symbolic.cos(th1)
  c2=pydrake.symbolic.cos(th2)
  c21=pydrake.symbolic.cos(th2-th1)
  s21=pydrake.symbolic.sin(th2-th1)
  spring=-k*(kL-z0+uX)

  prog=MathematicalProgram()

  ddx0=prog.NewContinuousVariables(1,"ddx0")
  ddy0=prog.NewContinuousVariables(1,"ddy0")
  ddth1=prog.NewContinuousVariables(1,"ddth1")
  ddz0=prog.NewContinuousVariables(1,"ddz0")
  ddth2=prog.NewContinuousVariables(1,"ddth2")

  print((c1*(m2*W*z0+I1)*ddth1 + m2*r2*W*c2*ddth2 + m2*W*ddx0 + m2*W*s1*ddz0)[0])
  print(W*m2*(dth1**2*W*s1 - 2*dth1*dz0*c1 + r2*dth2**2*s2 + r1*dth1**2*s2) - r1*Fx*c1**2 + (r1*Fy*s1-uT)*c1 + spring*W*s1)

  prog.AddConstraint( (c1*(m2*W*z0+I1)*ddth1 + m2*r2*W*c2*ddth2 + m2*W*ddx0 + m2*W*s1*ddz0)[0] == (W*m2*(dth1**2*W*s1 - 2*dth1*dz0*c1 + r2*dth2**2*s2 + r1*dth1**2*s2) - r1*Fx*c1**2 + (r1*Fy*s1-uT)*c1 + spring*W*s1)[0])
  prog.AddConstraint(-s1*(m2*W*z0+I1)*ddth1 - m2*r2*W*s2*ddth2 +m2*W*ddy0 +m2*W*c1*ddz0 
                         == W*m2*(dth1**2*W*c1 + 2*dth1*dz0*s1 + r2*dth2**2*c2 + r1*dth1**2*c1 - g) + r1*Fx*c1*s1
                         -(r1*Fy*c1-uT)*s1 + spring*W*c1)
  prog.AddConstraint( c1*(m1*r1*W-I1)*ddth1 +m1*W*ddx0 
                         ==W*(m1*r1*dth1**2*s1 - spring*s1 + Fx) - c1*(Fy*r1*s1-Fx*r1*c1-uT))
  prog.AddConstraint(-s1*(m1*r1*W-I1)*ddth1 +m1*W*ddy0 
                         ==W*(m1*r1*dth1**2*c1 - spring*c1 + Fy - m1*g) - s1*(Fy*r1*s1 -Fx*r1*c1-uT))
  prog.AddConstraint(-c21*I1*r2*ddth1 + I2*z0*ddth2 
                         == W*(spring*r2*s21+uT) - r2*c21*(r1*Fy*s1 - r1*Fx*c1 - uT))
  solver=SnoptSolver()
  result=solver.solve(prog)
  ddx0_r=result.GetSolution(ddx0)
  ddy0_r=result.GetSolution(ddy0)
  ddth1_r=result.GetSolution(ddth1)
  ddz0_r=result.GetSolution(ddz0)
  ddth2_r=result.GetSolution(ddth2)
  dstate=[dx0, dy0, dth1, dz0, dth2, ddx0_r, ddy0_r, ddth1_r, ddz0_r, ddth2_r]
  return dstate

Helper function for computing forward euler:

In [0]:
def forward_euler(state,dstate,dt):
    for i,q in enumerate(state):
      state_next[i] = state[i] + dstate[i]*dt
    return state_next

Below is vector system for intaking current state and control efforts and outputing the state at the next time step

In [0]:
class Hopper_Dyanmics(VectorSystem):
  def __init__(self, dt):
    #13 inputs: 10 state variables, 2 inputs (torque and position), one bool (in air)
    #10 outputs: state at next step

    VectorSystem.__init__(self,13,10)
    
    #set dt for euler forward approx for state
    self.dt=dt

  def DoCalcVectorOutput(self, context, state_uX_uT_in_air, unused, next_state):
    state = state_uX_uT_in_air[0:9]
    uX=state_uX_uT_in_air[10]
    uT=state_uX_uT_in_air[11]
    in_air=state_uX_uT_in_air[12]

    dstate=calc_dstate(state,uX,uT,in_air)

    next_state[:] = forward_euler(state,dstate,self.dt)

In [55]:
prog=MathematicalProgram()
N=50
dt=0.01
time_steps=int(N/dt)

state=prog.NewContinuousVariables(N+1,10,'state')
lambdas=prog.NewContinuousVariables(N,2,'lambdas')
uX=prog.NewContinuousVariables(N,1,'uX')
uT=prog.NewContinuousVariables(N,1,'uT')
grounded=prog.NewBinaryVariables(N,1,'grounded')

M=1000000

#print(state[0])


for t in range(time_steps):
  prog.AddConstraint(state[t+1] == forward_euler(state[t], calc_dstate(state[t],uX[t],uT[t],lambdas[t]),dt)) #dynamics constraint
  
  #forces are > 0 (Fx can be negative, so abs(Fx)>0)
  prog.AddConstraint(abs(lambdas[t,0])>=0)
  prog.AddConstraint(lambdas[t,1]>=0)

  #y0 is always above ground. using get_ground_height so that we can change ground geometry
  prog.AddConstraint((state[t,1]-get_ground_height(state[t,0]))>=0)

  #below is complimentarity constraint
  prog.AddConstraint(abs(lambdas[t,0])<=grounded[t]*M)
  prog.AddConstraint(lambdas[t,1]<=grounded[t]*M)
  prog.AddConstraint((state[t,1]-get_ground_height(state[t,0]))<=(1-grounded[t])*M)
  
  #add controller cost
  prog.AddCost(uX[t]**2)
  prog.AddCost(uT[t]**2)
  prog.AddCost(state[t].dot(state[t]))

solver=SnoptSolver()
result=solver.Solve(prog)


(10 * (ddx0(0) * (-0.5 + state(0,3))) + (ddth1(0) * (10 + 10 * (state(0,3) * (-0.5 + state(0,3)))) * cos(state(0,2))) + 10 * (ddz0(0) * (-0.5 + state(0,3)) * sin(state(0,2))) + 7 * (ddth2(0) * (-0.5 + state(0,3)) * cos(state(0,4))))
[<Expression "( - 0.5 * (lambdas(0,0) * pow(cos(state(0,2)), 2)) + 10 * ((-0.5 + state(0,3)) * ( - 2 * (state(0,7) * state(0,8) * cos(state(0,2))) + (pow(state(0,7), 2) * (-0.5 + state(0,3)) * sin(state(0,2))) + 0.5 * (pow(state(0,7), 2) * sin(state(0,4))) + 0.69999999999999996 * (pow(state(0,9), 2) * sin(state(0,4))))) - 1000 * ((-0.5 + state(0,3)) * (1 - state(0,3) + uX(0)) * sin(state(0,2))) + (( - uT(0) + 0.5 * (lambdas(0,1) * sin(state(0,2)))) * cos(state(0,2))))">]


RuntimeError: ignored

In [30]:
prog=MathematicalProgram()
#define state variables

#and their derivatives
dx0=prog.NewIndeterminates(1,'dx0')
dy0=prog.NewIndeterminates(1,'dy0')
dth1=prog.NewIndeterminates(1,'dth1')
dz0=prog.NewIndeterminates(1,'dz0')
dth2=prog.NewIndeterminates(1,'dth2')

uX=prog.NewIndeterminates(1,'uX')
uT=prog.NewIndeterminates(1,'uT')

  #get contact forces
Fx=prog.NewIndeterminates(1,'Fy')
Fy=prog.NewIndeterminates(1,'Fx')

  #define constants
r1=0.5
r2=0.7
m1=10
m2=10
I1=10
I2=10
k=1000
kL=1 #relaxed length
g=9.81

  #intermediates:
W=z0-r1
print(th1)
s1=prog.NewIndeterminates(1,'s1')
s2=prog.NewIndeterminates(1,'s2')
c1=prog.NewIndeterminates(1,'c1')
c2=prog.NewIndeterminates(1,'c2')
c21=prog.NewIndeterminates(1,'c21')
s21=prog.NewIndeterminates(1,'s21')
spring=-k*(kL-z0+uX)


ddx0=prog.NewContinuousVariables(1,"ddx0")
ddy0=prog.NewContinuousVariables(1,"ddy0")
ddth1=prog.NewContinuousVariables(1,"ddth1")
ddz0=prog.NewContinuousVariables(1,"ddz0")
ddth2=prog.NewContinuousVariables(1,"ddth2")

print(s1**2+c1**2)#

#prog.AddConstraint()
#prog.AddConstraint(eq(s2**2+c2**2,1))
#prog.AddConstraint(eq(s21**2+c21**2,1))


prog.AddConstraint( eq(c1*(m2*W*z0+I1)*ddth1 + m2*r2*W*c2*ddth2 + m2*W*ddx0 + m2*W*s1*ddz0, W*m2*(dth1**2*W*s1 - 2*dth1*dz0*c1 + r2*dth2**2*s2 + r1*dth1**2*s2) - r1*Fx*c1**2 + (r1*Fy*s1-uT)*c1 + spring*W*s1))
prog.AddConstraint(-s1*(m2*W*z0+I1)*ddth1 - m2*r2*W*s2*ddth2 +m2*W*ddy0 +m2*W*c1*ddz0 
                         == W*m2*(dth1**2*W*c1 + 2*dth1*dz0*s1 + r2*dth2**2*c2 + r1*dth1**2*c1 - g) + r1*Fx*c1*s1
                         -(r1*Fy*c1-uT)*s1 + spring*W*c1)
prog.AddConstraint( c1*(m1*r1*W-I1)*ddth1 +m1*W*ddx0 
                         ==W*(m1*r1*dth1**2*s1 - spring*s1 + Fx) - c1*(Fy*r1*s1-Fx*r1*c1-uT))
prog.AddConstraint(-s1*(m1*r1*W-I1)*ddth1 +m1*W*ddy0 
                         ==W*(m1*r1*dth1**2*c1 - spring*c1 + Fy - m1*g) - s1*(Fy*r1*s1 -Fx*r1*c1-uT))
prog.AddConstraint(-c21*I1*r2*ddth1 + I2*z0*ddth2 
                         == W*(spring*r2*s21+uT) - r2*c21*(r1*Fy*s1 - r1*Fx*c1 - uT))
  


[Variable('th1(0)', Continuous)]
[<Expression "(pow(s1(0), 2) + pow(c1(0), 2))">]


RuntimeError: ignored